## Red Pass Green Pass Classifier

Model training, using CNN, with Tensorflow Keras

In [ ]:
import os 
import random
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# from keras.callbacks import EarlyStopping

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Loading of All Images in a Folder

In [ ]:
# Reads in images as long as of image format
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            filenames.append(filename)
    return (images, filenames)

### Training the CNN model

In [ ]:
# Train the Model
def train_red_green_model(processed_green_path, processed_red_path):
    positive_samples = load_images_from_folder(processed_green_path)
    negative_samples = load_images_from_folder(processed_red_path)
    image_size = positive_samples[0][0].shape
    print('Image Dimensions: ' + str(image_size))
    image_height = image_size[0]
    image_width = image_size[1]
    image_colour = image_size[2]
    print('Number of positive sampels: ' + str(len(positive_samples[0])))
    print('Number of negative sampels: ' + str(len(negative_samples[0])))

    # 0 = Red, 1 = Green
    # Can look at divding the RGB value by 255 here to fit values between 0 and 1
    training_data = []
    for sample in positive_samples[0]:
        training_data.append([sample, 1])
    for sample in negative_samples[0]:
        training_data.append([sample, 0])

    # Shuffle training data to ensure better results
    random.shuffle(training_data)

    X = [] # All Training Data
    y = [] # All Labels
    for image, label in training_data:
        X.append(image)
        y.append(label)

    # Convert into numpy array
    X = np.array(X).reshape(-1, image_height, image_width, 3)
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    # Format into np.array type as Keras model does not accept other types
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # CNN Model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(image_height, image_width, image_colour)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Need to look at early stopping
    model.fit(X_train, y_train, epochs=10, batch_size=1, validation_data=(X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)
    print('Loss: ' + str(loss) + '    ' + 'Accuracy: ' + str(accuracy))

    # Save the model
    # model.save('GreenRedClassifier.model')
    print('Model Trained and Saved!')


In [ ]:
# Images must have been already pre-processed with the "ImageFormatter" script before proceeding
train_red_green_model(processed_green_path='./ProcessedImages/GreenPass', processed_red_path='./ProcessedImages/RedPass')